### 上个Demo中,使用了Softmax Regression模型进行mnist数据集的分类. 这是最简单的模型,但它与神经网络的最大区别是没有隐含层,因此拟合能力不足. 本例程开始搭建具有隐含层的神经网络.
### 神经网络在使用中常常会出现下面几个问题, 因此需要添加一些参数进行调节. 本例程会把这些参数都添加进去,尽可能提高神经网络的分类准确率.
#### 1.过拟合. 表示模型在训练时分类准确度高,在测试集上准确度较低, 推广能力差的情况. 可以引入dropout参数, 随机去除训练样本中的某些特征,可理解为对特征的一种采样.
#### 2.参数调试困难. 神经网络不是凸优化问题,它处处充满了局部最优, 不同的学习速率常常会导致落入截然不同的局部最优中. 我们希望起初学习速率大一些, 加速收敛;后期学习速率小一些,可以稳定落入一个局部最优解. 为此设计出了Adagrad, Adam等自适应方法, 经实验都比SGD效果要好.
#### 3.梯度弥散问题. 之前神经网络最常用sigmoid激活函数, 但在多层的传递后梯度会急剧减小,训练效果很差. 为此引入了Relu函数, 当某个神经元的信号在超过某个阈值时才被激活,平常处于被抑制状态.这样即使隐含层数很多,也不会出现梯度弥散问题.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_DATA",one_hot = True)
    
    

Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz


In [7]:
# 创建会话.后续过程都在该会话中进行
sess = tf.InteractiveSession()

input_size = 784  # 输入神经元的个数,等于输入数据的特征维数
hidden_size = 300  # 隐含层神经元的个数,需要调参取最优值

# 隐含层的参数
# 因为隐含层使用激活函数Relu,需要避免零梯度现象,不能将初始值赋为0.此处使用正态分布,标准差设为0.1
W1 = tf.Variable(tf.truncated_normal([input_size,hidden_size],stddev=0.1))
b1 = tf.Variable(tf.zeros([hidden_size]))

# 输出层的参数.输出层仍采用Softmax,初始值仍全赋为0即可.
W2 = tf.Variable(tf.zeros([hidden_size,10]))
b2 = tf.Variable(tf.zeros([10]))

x = tf.placeholder(tf.float32,[None,input_size])

# 优化1:加入dropout(数值表示需要保留节点的概率).要注意的是,dropout在训练和测试时是不同的.
# 训练时为防止过拟合,应当小于1;测试时需要用到全部特征,应当等于1.因此也需要一个占位符,在训练和测试时可灵活调节
drop_out = tf.placeholder(tf.float32)

# 定义隐含层的模型结构.
# 优化2:加入relu.公式为:hidden = relu(w1*x+b1).再加上dropout.
hidden = tf.nn.relu(tf.matmul(x,W1) + b1)
hidden_drop = tf.nn.dropout(hidden,drop_out)

# 输出层的模型结构与上个例程的softmax相同
y = tf.nn.softmax(tf.matmul(hidden_drop,W2) + b2)

# 定义loss函数.优化的目标是使该函数值尽量减小.使用常用的交叉熵作为损失函数.设置y_即真正的label值,用来计算交叉熵
y_ = tf.placeholder(tf.float32,[None,10])
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))

# 优化3:将SGD寻优算法改为Adam.learning_rate要通过不断调试才能找到最合适的值.
#       最初设置的为0.5,效果特别差.
#       笔者也测试了Adagrad算法,learning_rate取0.3时效果几乎一致.
learning_rate = 0.001
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# 全局参数初始化
tf.global_variables_initializer().run()

# 设置准确率函数.当预测值与实际值相同时,即为分类准确.然后求整体的准确率
correct_pred = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
acc = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

# 喂参数,开始训练.每次随机抽取100条样本进行训练,共迭代1000次.注意:输入的y值是真实标签,应当是y_! y是输出值
for i in range(10000):
    batch_x, batch_y = mnist.train.next_batch(100)
    optimizer.run({x:batch_x,y_:batch_y,drop_out:0.75})   #训练时drop_out设置为0.75

    if i%500 == 0:
        print "迭代"+ str(i) + "次,训练准确率为:" + str(acc.eval({x:batch_x,y_:batch_y,drop_out:0.75}))
        
#     如上的run()操作也可写为:sess.run(optimizer,feed_dict={x:batch_x,y_:batch_y})
#                        与:sess.run(acc,feed_dict={x:batch_x,y_:batch_y})
#     这样看起来更清晰.在执行算法定义的代码时,计算并没有发生.只有调用run方法并feed数据时才真正执行.


print "整体测试准确率为:" + str(acc.eval({x:mnist.test.images,y_:mnist.test.labels,drop_out:1.0}))   #测试时drop_out设置为1.0

迭代0次,训练准确率为:0.53
迭代500次,训练准确率为:0.89
迭代1000次,训练准确率为:0.93
迭代1500次,训练准确率为:0.98
迭代2000次,训练准确率为:0.97
迭代2500次,训练准确率为:0.99
迭代3000次,训练准确率为:0.95
迭代3500次,训练准确率为:0.97
迭代4000次,训练准确率为:0.98
迭代4500次,训练准确率为:0.93
迭代5000次,训练准确率为:0.98
迭代5500次,训练准确率为:0.99
迭代6000次,训练准确率为:1.0
迭代6500次,训练准确率为:1.0
迭代7000次,训练准确率为:0.98
迭代7500次,训练准确率为:1.0
迭代8000次,训练准确率为:0.98
迭代8500次,训练准确率为:1.0
迭代9000次,训练准确率为:1.0
迭代9500次,训练准确率为:1.0
整体测试准确率为:0.9784
